In [1]:
import numpy as np
import kpm
import wget
import os

In [2]:
url = 'https://www.emilyjgriffith.com/s/'
#if(os.path.isfile('lnqs.npy')==False): wget.download(url+'lnqs.npy') # W22 q vectors
#if(os.path.isfile('lnAs.npy')==False): wget.download(url+'lnAs.npy') # W22 A vectors
#if(os.path.isfile('alldata_train.npy')==False): wget.download(url+'alldata_train.npy') # APOGEE stellar abundances
#if(os.path.isfile('allivars_train.npy')==False): wget.download(url+'allivars_train.npy') # APOGEE stellar abundance inverse variance

In [3]:
elements  = np.array(['Mg','O','Si','S','Ca','CN','Na','Al','K','Cr','Fe','Ni','Mn','Co','Ce'])
alldata = np.load('alldata_train.npy')
allivars = np.load('allivars_train.npy')

data = kpm.AbundData(elements, alldata, allivars)

In [10]:
kpm_init = kpm.FixedParams(data, K=1)

In [11]:
kpm_init.K = 1

In [6]:
knot_xs = np.array([[-0.8, -0.5, -0.4, -0.3, -0.2, -0.1, 0.0, 0.1, 0.2, 0.3, 0.6],
                    [-0.8, -0.5, -0.4, -0.3, -0.2, -0.1, 0.0, 0.1, 0.2, 0.3, 0.6]])

In [7]:
# kpm_init.knot_xs = knot_xs

In [12]:
ad, fit = kpm.initialize_2(data, kpm_init)

initialize_2(): []
initialize_2(): []


In [16]:
for i in range(2):
    fit = kpm.Aq_step(ad, kpm_init, fit)
    print(i + 1)

9935339.0 10137286.0 6706973.5 6706973.0 6226720.0
Aq-step(): we took a step! -5 6226720.0 3708619.0
1
6226720.0 6234439.0 6206545.0 6206544.5 6201061.5
Aq-step(): we took a step! -5 6201061.5 25658.5
2
